In [ ]:
import socket

# Get the hostname of the local computer
hostname = socket.gethostname()

# Get the IP address of the local computer
ip_address = socket.gethostbyname(hostname)

# Print the hostname and IP address
print("Hostname:", hostname)
print("IP Address:", ip_address)


In [ ]:
# !pip install paramiko
import paramiko

class SSHServer:
    def __init__(self, host, username, password):
        self.host = host
        self.username = username
        self.password = password

    def start(self):
        self.server = paramiko.Transport((self.host, 22))
        self.server.add_server_key(paramiko.RSAKey.generate(2048))
        self.server.start_server(server=self)

    def check_auth_password(self, username, password):
        if (username == self.username) and (password == self.password):
            return paramiko.AUTH_SUCCESSFUL
        else:
            return paramiko.AUTH_FAILED

    def check_channel_request(self, kind, chanid):
        if kind == "session":
            return paramiko.OPEN_SUCCEEDED
        else:
            return paramiko.OPEN_FAILED_ADMINISTRATIVELY_PROHIBITED

    def start_session(self, channel):
        channel.send("Welcome to the server!\n")
        while True:
            command = channel.recv(1024).decode().strip()
            if command == "exit":
                channel.send("Goodbye!\n")
                break
            else:
                output = self.execute_command(command)
                channel.send(output)

    def execute_command(self, command):
        # execute the command on the server and return the output
        # this method should be implemented by the subclass
        return ""

class SSHClient:
    def __init__(self, host, username, password):
        self.host = host
        self.username = username
        self.password = password

    def connect(self):
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(self.host, username=self.username, password=self.password)

    def execute_command(self, command):
        stdin, stdout, stderr = self.client.exec_command(command)
        output = stdout.read().decode()
        return output

    def send_command(self, command):
        channel = self.client.invoke_shell()
        channel.send(command + "\n")
        output = ""
        while not channel.recv_ready():
            pass
        while channel.recv_ready():
            output += channel.recv(1024).decode()
        return output

    def close(self):
        self.client.close()


In [ ]:
# Create an instance of the server and start it
server = SSHServer(ip_address, hostname, 'password')
server.start()

In [ ]:
# Create an instance of the client and connect to the server
client = SSHClient(ip_address, hostname, "password")
client.connect()

# Execute a command on the server and print the output
output = client.execute_command("ls -l")
print(output)

# Send a command to the server and print the output
output = client.send_command("echo 'hello, world!'")
print(output)

# Close the SSH connection
client.close()


In [ ]:
import subprocess

# Replace "example.com" with the IP address or hostname of your remote host.
host = "192.168.42.18"
username = "pi"
password = "Blkioq#7"
command = "ls -l"

# Connect to the remote host using SSH.
ssh = subprocess.Popen(["ssh", f"{username}@{host}", command],
                       stdin=subprocess.PIPE,
                       stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE,
                       universal_newlines=True,
                       bufsize=0)

# Enter the password if prompted.
if ssh.stderr:
    password_prompt = ssh.stderr.read()
    print(password_prompt)
    if "password" in password_prompt.lower():
        ssh.stdin.write(password + "\n")
        ssh.stdin.flush()

# Print the output of the command.
for line in ssh.stdout:
    print(line.strip())


In [ ]:
import subprocess

class SSHClient:
    def __init__(self, host, username, password):
        self.host = host
        self.username = username
        self.password = password

    def connect(self):
        self.process = subprocess.Popen(
            [
                "ssh",
                "-o", "ServerAliveInterval=60",
                f"{self.username}@{self.host}",
            ],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        self.process.stdin.write(f"{self.password}\n".encode())
        self.process.stdin.flush()

    def execute(self, command):
        self.process.stdin.write(f"{command}".encode())
        self.process.stdin.flush()
        output = self.process.stdout.read().decode()
        return output

    def disconnect(self):
        self.process.stdin.close()
        self.process.stdout.close()
        self.process.stderr.close()
        self.process.terminate()


In [ ]:
# !ulimit -n 2048

client = SSHClient('192.168.42.18', 'pi', 'pi')

In [ ]:
client.connect()

In [ ]:
client.execute('ls -l')

In [ ]:
client.disconnect()

In [ ]:
client.process.stdout.read().decode()

In [ ]:
import os
os.system('ssh pi@192.168.42.18')